# **LlamaIndex: Data indexing/retrieval for RAG**

### This specific example is using the chat engine in context mode.

Chat engine is a high-level interface for having a conversation with your data (multiple back-and-forth instead of a single question & answer).
Think ChatGPT, but augmented with your knowledge base.

*For each chat interaction:*

- first retrieve text from the index using the user message
- set the retrieved text as context in the system prompt
- return an answer to the user message


This approach is simple, and works for questions directly related to the knowledge base and general interactions.

# **1. Install the necessary packages**

In [ ]:
!pip install llama-index llama-index-vector-stores-pinecone llama-index-llms-replicate

## **2. Read documents, index them, and save them to a storage directory**

In [ ]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import settings
from llama_index.embeddings.openai import OpenAIEmbedding

os.environ["OPENAI_API_KEY"] = (
    "your_openai_api_key"
)
settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large", dimensions=1536)

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist("storage") # To avoid reprocessing documents every time, you can persist the index to disk:

## **3. Load index from storage and initialize chat engine**

In [89]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(storage_context)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as answering natural language questions about GolfGuiders application manual and Interview questions."
    ),
)

Loading llama_index.core.storage.kvstore.simple_kvstore from storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage\index_store.json.


## **4. Chat with your data**

In [91]:
response = chat_engine.chat("How do I answer 'Where do you see yourself five years from now?'")
print(response)

When answering the question "Where do you see yourself five years from now?" in an interview, you want to demonstrate your commitment to the position and the company, as well as your ambition and realistic career planning. Here’s how to craft your response:

1. **Long-Term Commitment**: Start by reassuring the interviewer that you are looking to make a long-term commitment to the company. Express that the position aligns with what you are looking to do and are well-qualified to handle.

2. **Focus on the Present**: Emphasize that your immediate goal is to excel in the position you're applying for, and you believe that future opportunities will naturally arise from your performance.

3. **Avoid Specific Titles**: Avoid naming specific promotions or titles you wish to achieve, as this can come across as presumptuous. Instead, focus on the skills and experiences you hope to gain.

4. **Confidence in Growth**: Convey confidence that by working hard and performing well, opportunities will o

# **5. Configure the chat engine with custom llm**

In [92]:
from llama_index.core import StorageContext, load_index_from_storage, Settings
from llama_index.llms.openai import OpenAI
# from llama_index.llms.replicate import Replicate

# change the global default LLM
llm = OpenAI(model="gpt-4o", temperature=0.7)
# llm = Replicate(model="meta/meta-llama-3-8b-instruct", temperature=0.7)

Settings.llm = llm

storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(storage_context)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    llm=llm,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as answering natural language questions about GolfGuiders application manual."
    ),
)

Loading llama_index.core.storage.kvstore.simple_kvstore from storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage\index_store.json.


# **6. Streaming responses**

In [94]:
response = chat_engine.stream_chat("Can you work under pressure?")
for token in response.response_gen:
    print(token, end="")

Absolutely. I thrive under pressure and have successfully accomplished various goals and projects in high-pressure situations. For instance, I managed a project with a tight deadline where I had to coordinate with multiple teams to ensure timely delivery, and we achieved our objectives efficiently despite the challenges. This experience highlighted my ability to remain focused and productive even when the stakes are high.

# **7. Context Retriever**

In [95]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(storage_context)
retriever = index.as_retriever(similarity_top_k=3)

Loading llama_index.core.storage.kvstore.simple_kvstore from storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage\index_store.json.


In [96]:
# Here is the behinde the scenes of the retriever how it retrieve relevent chunks
nodes = retriever.retrieve("How do I answer 'Where do you see yourself five years from now?'")

for i, node in enumerate(nodes):
    print(f"Node {i+1}: Score: {node.score}\nText: {node.text}\n")

Node 1: Score: 0.8292969604066961
Text: “I could also help you in many things they don’t teach at the Harvard Business School.  
For example…(how to hire, train, motivate, etc.)  When it comes to knowing how to work 
well with people and getting the most out of them, there’s just no substitute for what you 
learn over many years of front-line experience.  You company would gain all this, too.” 
“From my side, there are strong benefits, as well.   Right now, I am unemployed.  I want 
to work, very much, and the position you have here is exactly what I love to do and am 
best at.  I’ll be happy doing this work and that’s what matters most to me, a lot more that 
money or title.” 
“Most important, I’m looking to make a long term commitment in my career now. I’ve had 
enough of job-hunting and want a permanent spot at this point in my career.  I also know 
that if I perform this job with excellence, other opportunities cannot help but open up for 
me right here.  In time, I’ll find many ot

In [97]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever=retriever)
response = query_engine.query("How can I create a new post?")
print(response)

To create a new post, start by tapping the "Got something to say? +" field next to your profile picture on the Home Screen. This will open the Post Creation Screen where you can customize your post and publish it. You have various options to enhance your post, such as adding text, images, videos, tagging people, sharing your mood or activity, choosing a background color, and attaching music. You can also set the visibility of your post to Public, Private, or Circle.
